In [1]:
from datetime import datetime, timedelta

import pandas as pd
from database import Database
from email_sender import EmailSender
from excel_writer import ExcelWriter
from sqlalchemy import select

In [2]:
db = Database()

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
date_fr = datetime.now().date()
date_to = date_fr + timedelta(days=6)
date_fr, date_to

(datetime.date(2024, 2, 18), datetime.date(2024, 2, 24))

In [4]:
select_bookings = (
    select(
        db.clients_operator.c.short_name.label("operator_name"),
        db.gwg_reservation.c.bkg_ref,
        db.mapping_hotel.c.external_name.label("hotel_name"),
        db.gwg_reservation.c.guest_name,
        db.gwg_reservation.c.sales_date,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.out_date,
        db.gwg_reservation.c.room_type,
        db.gwg_reservation.c.meal,
        db.gwg_reservation.c.adult,
        db.gwg_reservation.c.child,
        db.gwg_reservation.c.days,
    )
    .outerjoin(
        db.mapping_hotel,
        db.gwg_reservation.c.hotel_id == db.mapping_hotel.c.external_code,
    )
    .outerjoin(
        db.mapping_operator,
        db.gwg_reservation.c.operator_id == db.mapping_operator.c.external_code,
    )
    .outerjoin(db.clients_operator)
    .where(
        db.gwg_reservation.c.hotel_id.in_([202191, 195820, 193348, 193352]),
        db.gwg_reservation.c.in_date.between(date_fr, date_to),
        db.gwg_reservation.c.status != "Can",
    )
    .order_by(
        db.mapping_hotel.c.external_name,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.ref_id,
    )
)

In [5]:
df = pd.read_sql(sql=select_bookings, con=db.engine.connect())

In [6]:
df.shape

(116, 12)

In [7]:
df.head()

,operator_name,bkg_ref,hotel_name,guest_name,sales_date,in_date,out_date,room_type,meal,adult,child,days
0,FTIDE,LLDPW,FANAR HOTEL & RESIDENCES,MARIA HAUSINGER,2023-09-02,2024-02-18,2024-03-02,Twin Superior,AI,2,0,13
1,BIGDE,4603908,FANAR HOTEL & RESIDENCES,MORITZ JUHNKE,2023-09-11,2024-02-18,2024-02-28,Deluxe Room,AI,2,0,10
2,BIGDE,4603904,FANAR HOTEL & RESIDENCES,DANIEL JUHNKE,2023-09-11,2024-02-18,2024-02-28,Deluxe Room,AI,2,0,10
3,FTIDE,CTR3E,FANAR HOTEL & RESIDENCES,PHILIPP GAJECK,2023-09-26,2024-02-18,2024-02-28,Deluxe Room,AI,2,0,10
4,FTIDE,L45FS,FANAR HOTEL & RESIDENCES,"JOS HIMMETSBERGER, WERNER JOS",2023-10-25,2024-02-18,2024-02-24,Deluxe Suite,AI,2,1,6


In [8]:
outfile = f'Orascom_{date_fr.strftime("%Y_%m_%d")}_{date_to.strftime("%Y_%m_%d")}.xlsx'

In [9]:
excel_writer = ExcelWriter([df], outfile)
excel_writer.write_to_excel(date_column=[4, 5, 6])
excel_writer.writer.close()

In [12]:
from email_sender import EmailSender

sender = EmailSender(
    subject="Orascom Weekly Arrival",
    to=["analyst-contracting@meetingpointuae.com"],
)

sender.send_email(
    [outfile],
    """
        <p>This is an automated report.</p>
        <p>Please do not reply.</p>
    """,
)